In [1]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from numpy import load
from torch.utils.data import Dataset

In [2]:
mbrnlist1=[(5,194),(2,155),(12,100),(17,29),(42,1),(44,1),(50,92),(2,83),(4,10118),(8,298)]
mbrnlist2=[(5,194),(12,100),(2,155),(17,29),(42,1),(44,1),(2,83),(4,10118),(4,9997),(50,91)]

mbrnlist=mbrnlist1+mbrnlist2
mbrnlist=set(mbrnlist)
mbrnlist=list(mbrnlist)

# load array
MBR_NO,BRN_NO=mbrnlist[11]
# print the array

In [3]:
Data_train = load('Data0930_'+str(MBR_NO)+'_'+str(BRN_NO)+'norm_train.npy',allow_pickle=True)
Data_test =  load('Data0930_'+str(MBR_NO)+'_'+str(BRN_NO)+'norm_test.npy',allow_pickle=True)


Xdata=[]
Ydata=[]
Xtrain_data=[]
Ytrain_data=[]
Xtest_data=[]
Ytest_data=[]



for idx in range(len(Data_train)//39):
    if(np.isinf(Data_train[39*idx:39*(idx+1)][:,:].tolist()).any()):
        print(np.isinf(Data_train[39*idx:39*(idx+1)][:,:].tolist()).any())
        continue
    Xtrain_data.append(Data_train[39*idx:39*(idx+1)][:,:].tolist())
    Ytrain_data.append(Data_train[39*idx:39*(idx+1)][:,-1].tolist())
for idx in range(len(Data_test)//39):
    if(np.isinf(Data_test[39*idx:39*(idx+1)][:,:].tolist()).any()):
        print(np.isinf(Data_test[39*idx:39*(idx+1)][:,:].tolist()).any())
        continue
    Xtest_data.append(Data_test[39*idx:39*(idx+1)][:,:].tolist())
    Ytest_data.append(Data_test[39*idx:39*(idx+1)][:,-1].tolist())
    
Xtrain_data=np.vstack(Xtrain_data)
Ytrain_data=np.vstack(Ytrain_data)

Xtrain_data=torch.FloatTensor(Xtrain_data)
Ytrain_data=torch.IntTensor(Ytrain_data)
Ytrain_data=Ytrain_data.view(-1)


Xtest_data=np.vstack(Xtest_data)
Ytest_data=np.vstack(Ytest_data)
Xtest_data=torch.FloatTensor(Xtest_data)
Ytest_data=torch.IntTensor(Ytest_data)
Ytest_data=Ytest_data.view(-1)

Ytrain_data=2*(Ytrain_data>0).long()+(Ytrain_data==0).long()
Ytest_data=2*(Ytest_data>0).long()+(Ytest_data==0).long()
Ytrain_data=Ytrain_data.T
Ytest_data=Ytest_data.T

Ydata=(torch.stack([Ydata>0,Ydata==0, Ydata<0],axis=0)).float()

In [4]:
Xtrain_data.size()

torch.Size([74100, 7])

In [5]:
Ytrain_data

tensor([1, 1, 1,  ..., 1, 1, 1])

In [6]:
Xtrain_data

tensor([[ 2.5800e+04,  3.8780e-01,  1.7981e-01,  ...,  2.9070e-02,
         -9.6899e-03,  0.0000e+00],
        [ 2.5800e+04,  6.1143e-01,  2.2340e-01,  ...,  2.9070e-02,
         -9.6899e-03,  0.0000e+00],
        [ 2.5800e+04,  7.1929e-01,  2.2576e-01,  ...,  2.9070e-02,
         -9.6899e-03,  0.0000e+00],
        ...,
        [ 2.8250e+03,  2.8344e-01,  1.0433e-01,  ...,  3.8938e-02,
         -3.5398e-03,  0.0000e+00],
        [ 2.8250e+03,  2.7932e-01,  1.0208e-01,  ...,  3.8938e-02,
         -3.5398e-03,  0.0000e+00],
        [ 2.8250e+03,  2.5743e-01,  1.1438e-01,  ...,  3.8938e-02,
         -3.5398e-03,  0.0000e+00]])

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



def batchify(data, bsz,bptt):
    # 데이터셋을 bsz 파트들로 나눕니다.
    if(data.size(0)%(bsz*bptt)!=0):
        
#         print(data.size(0),bsz,bptt)
        
        data=data.view(-1,bptt,data.size(1)).transpose(0,1).contiguous()
        return data.to(device)
    nbatch = data.size(0) // bsz
    # 깔끔하게 나누어 떨어지지 않는 추가적인 부분(나머지들) 은 잘라냅니다.
    data = data.narrow(0, 0, nbatch * bsz)
    # 데이터에 대하여 bsz 배치들로 동등하게 나눕니다.
    data = data.view(bsz, -1,data.size(1)).transpose(0,1).contiguous()
    return data.to(device)


In [8]:


bptt = 39
def get_batch(source, i,bs):
    seq_len = min(bptt*bs, len(source)  - i)
    data = source[i:i+seq_len]
#     if(seq_len!=bptt*bs):
#         print(seq_len)
    target = source[i:i+seq_len].reshape(-1)
    return data, target


In [9]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

INPUT_SIZE=Xtrain_data.shape[1]
EMB_SIZE=128
HIDDEN_SIZE=128
NUM_LAYERS=6
PROJ_SIZE=3
BATCH_SIZE=32

In [10]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import numpy as np

def batch_matmul(seq, weight, nonlinearity=''):
    s = None
    for i in range(seq.size(0)):
        _s = torch.mm(seq[i], weight)
        if(nonlinearity=='tanh'):
            _s = torch.tanh(_s)
        _s = _s.unsqueeze(0)
        if(s is None):
            s = _s
        else:
            s = torch.cat((s,_s),0)
    return s.squeeze()

class RNNModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers,proj_size, dropout=0.5, tie_weights=False, attention=True,
                 attention_width=3, cuda=True):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
#         self.encoder = nn.Embedding(ntoken, ninp)
        self.encoder = nn.Linear(ntoken, ninp)
        if rnn_type in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, rnn_type)(ninp, nhid, nlayers, dropout=dropout)
        else:
            try:
                nonlinearity = {'RNN_TANH': 'tanh', 'RNN_RELU': 'relu'}[rnn_type]
            except KeyError:
                raise ValueError("""An invalid option for `--model` was supplied,
                                 options are ['LSTM', 'GRU', 'RNN_TANH' or 'RNN_RELU']""")
            self.rnn = nn.RNN(ninp, nhid, nlayers, nonlinearity=nonlinearity, dropout=dropout)
        if attention:
            self.decoder = nn.Linear(nhid, ntoken)
        else:
            self.decoder = nn.Linear(nhid, ntoken)
        if tie_weights:
            if nhid != ninp:
                raise ValueError('When using the tied flag, nhid must be equal to emsize')
            self.decoder.weight = self.encoder.weight

        self.softmax = nn.Softmax()
        if attention:
            self.AttentionLayer = AttentionLayer(cuda,nhid)
        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers
        self.attention = attention
        self.attention_width = attention_width
        self.generator = nn.Linear(nhid, proj_size)
    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, input):
        #print("input size:",input.size())
#         emb = self.drop(self.encoder(input))
        emb=self.encoder(input)
        #print("emb size:",emb.size())
        output, _ = self.rnn(emb)
        #print("rnn output",output.size())
        if self.attention:
            output = self.AttentionLayer.forward(output, self.attention_width)
        return self.generator(output)
        #
#         output = self.drop(output)
#         decoded = self.decoder(output.view(output.size(0) * output.size(1), output.size(2)))
#         return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        if self.rnn_type == 'LSTM':
            return (Variable(weight.new(self.nlayers, bsz, self.nhid).zero_()),
                    Variable(weight.new(self.nlayers, bsz, self.nhid).zero_()))
        else:
            return Variable(weight.new(self.nlayers, bsz, self.nhid).zero_())


class AttentionLayer(nn.Module):
    """Implements an Attention Layer"""

    def __init__(self, cuda, nhid):
        super(AttentionLayer, self).__init__()
        self.nhid = nhid
        self.weight_W = nn.Parameter(torch.Tensor(nhid,nhid))
        self.weight_proj = nn.Parameter(torch.Tensor(nhid, 1))
        self.softmax = nn.Softmax()
        self.weight_W.data.uniform_(-0.1, 0.1)
        self.weight_proj.data.uniform_(-0.1,0.1)
        self.cuda = cuda

    def forward(self, inputs, attention_width=3):
        results = None
        for i in range(inputs.size(0)):
            if(i<attention_width):
                output = inputs[i]
                output = output.unsqueeze(0)

            else:
                lb = i - attention_width
                if(lb<0):
                    lb = 0
                selector = torch.from_numpy(np.array(np.arange(lb, i)))
                if self.cuda:
                    selector = Variable(selector).cuda()
                else:
                    selector = Variable(selector)
                vec = torch.index_select(inputs, 0, selector)
                u = batch_matmul(vec, self.weight_W, nonlinearity='tanh')
                a = batch_matmul(u, self.weight_proj)
                a = self.softmax(a)
                output = None
                for i in range(vec.size(0)):
                    h_i = vec[i]
                    a_i = a[i].unsqueeze(1).expand_as(h_i)
                    h_i = a_i * h_i
                    h_i = h_i.unsqueeze(0)
                    if(output is None):
                        output = h_i
                    else:
                        output = torch.cat((output,h_i),0)
                output = torch.sum(output,0)
                output = output.unsqueeze(0)

            if(results is None):
                results = output

            else:
                results = torch.cat((results,output),0)

        return results

In [11]:
torch.manual_seed(0)



#lstm=Seq2SeqLSTM(input_size=INPUT_SIZE,hidden_size=HIDDEN_SIZE,num_layers=NUM_LAYERS,emb_size=EMB_SIZE,proj_size=PROJ_SIZE)

#lstm = RNNmodel(rnn_type='LSTM',ntoken=INPUT_SIZE,ninp=INPUT_SIZE,nhid=HIDDEN_SIZE,nlayers=NUM_LAYERS,emb_size=EMB_SIZE,proj_size=PROJ_SIZE)
lstm = RNNModel(rnn_type='LSTM',ntoken=INPUT_SIZE,ninp=EMB_SIZE,nhid=HIDDEN_SIZE,nlayers=NUM_LAYERS,proj_size=PROJ_SIZE,
                attention_width=38)
# for p in lstm.parameters():
#     if p.dim() > 1:
#         nn.init.xavier_uniform_(p)

lstm = lstm.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(lstm.parameters(), lr=1e-3, betas=(0.9, 0.98), eps=1e-9)

In [12]:
(Ytest_data==2).all()

tensor(False)

In [13]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer):
    model.train()
    losses = 0
    total=0
    correct=0
    correct1=0
    tot1=0
    correct2=0
    tot2=0
    correct0=0
    tot0=0
    conf00=0
    conf01=0
    conf02=0
    conf10=0
    conf11=0
    conf12=0
    conf20=0
    conf21=0
    conf22=0
    
    
#     for batch, i in enumerate(range(0, Xtrain_data.size(0) - 1, BATCH_SIZE*bptt)):
    i=0
    while(i<Xtest_data.size(0)):
        data, org_targets = get_batch(Xtrain_data, i,BATCH_SIZE)
        if (data.isnan().any() or data.isinf().any()):
#            print(data)
            continue
#         _,targets = get_batch(Ytrain_data,i)
        targets,_ = get_batch(Ytrain_data,i,BATCH_SIZE)
#         src = src.to(DEVICE)
#         tgt = tgt.to(DEVICE)
        targets=torch.unsqueeze(targets,1)
        src=batchify(data,BATCH_SIZE,bptt)
        tgt=batchify(targets,BATCH_SIZE,bptt)
        src=src[:-1]
        tgt=tgt[1:]
        logits = model(src)

        optimizer.zero_grad()

        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()
        
        _,predicted=torch.max(logits,-1)
        correct += (tgt.squeeze() == predicted).sum().item()
        total+=len(predicted)*BATCH_SIZE
        tot0+=(0== tgt.squeeze()).sum().item()
        tot1+=(1== tgt.squeeze()).sum().item()
        tot2+=(2== tgt.squeeze()).sum().item()
        correct0+=((0== predicted) &(0==tgt.squeeze())).sum().item()
        correct1+=((1== predicted)&(1==tgt.squeeze())).sum().item()
        correct2+=((2== predicted)&(2==tgt.squeeze())).sum().item()
        
        conf00+=((0== predicted) &(0==tgt.squeeze())).sum().item()
        conf01+=((0== predicted)&(1==tgt.squeeze())).sum().item()
        conf02+=((0== predicted)&(2==tgt.squeeze())).sum().item()
        conf10+=((1== predicted) &(0==tgt.squeeze())).sum().item()
        conf11+=((1== predicted)&(1==tgt.squeeze())).sum().item()
        conf12+=((1== predicted)&(2==tgt.squeeze())).sum().item()
        conf20+=((2== predicted) &(0==tgt.squeeze())).sum().item()
        conf21+=((2== predicted)&(1==tgt.squeeze())).sum().item()
        conf22+=((2== predicted)&(2==tgt.squeeze())).sum().item()

        
        
        
        i+=targets.size()[0]

    tp0=conf00
    fp0=conf01+conf02
    fn0=conf10+conf20
    if(tp0+fp0==0):
        prec0=0
    else:
        prec0=tp0/(tp0+fp0)
    if(tp0+fn0==0):
        reca0=0
    else:
        reca0=tp0/(tp0+fn0)
    
    tp1=conf11
    fp1=conf10+conf12
    fn1=conf01+conf21
    
    if(tp1+fp1==0):
        prec1=0
    else:
        prec1=tp1/(tp1+fp1)
    if(tp1+fn1==0):
        reca1=0
    else:
        reca1=tp1/(tp1+fn1)
    
    tp2=conf22
    fp2=conf20+conf21
    fn2=conf02+conf12
    
    if(tp2+fp2==0):
        prec2=0
    else:
        prec2=tp2/(tp2+fp2)
    if(tp2+fn2==0):
        reca2=0
    else:
        reca2=tp2/(tp2+fn2)
        
    prec=(prec0+prec1+prec2)/3
    reca=(reca0+reca1+reca2)/3
    f1sc=2*(prec*reca)/(prec+reca)
    print(total)
    print(correct)
    print("Acc:",correct/total)            
    print("Prec",prec)
    print("Recall",reca)
    print("F1",f1sc) 
    return losses / Xtrain_data.size(0),[conf00,conf01,conf02,conf10,conf11,conf12,conf20,conf21,conf22]

def evaluate(model):
    model.eval()
    losses = 0
    total=0
    correct=0
    correct1=0
    tot1=0
    correct2=0
    tot2=0
    correct0=0
    tot0=0
    conf00=0
    conf01=0
    conf02=0
    conf10=0
    conf11=0
    conf12=0
    conf20=0
    conf21=0
    conf22=0
    stime=time.time()
    with torch.no_grad():
        
        i=0
        while(i<Xtest_data.size(0)):
            data, org_targets = get_batch(Xtest_data, i,BATCH_SIZE)
            if (data.isnan().any() or data.isinf().any()):
                print(data)
                continue
            targets,_ = get_batch(Ytest_data,i,BATCH_SIZE)
            targets=torch.unsqueeze(targets,1)
            src=batchify(data,BATCH_SIZE,bptt)
            tgt=batchify(targets,BATCH_SIZE,bptt)
#             print(src.shape)
            if(src.shape[1]!=BATCH_SIZE):
                
                break
#             print(tgt.shape)
            src=src[:-1]
            tgt=tgt[1:]


            logits = model(src)
#            print(logits.reshape(-1, logits.shape[-1]).shape,'AA')
#            print(tgt.reshape(-1).shape,'BB')
            loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt.reshape(-1))
            if(loss.isnan()):
                print(src,tgt_input)
                break
            losses += loss.item()
            _,predicted=torch.max(logits,-1)

            
            correct += (tgt.squeeze() == predicted).sum().item()
            total+=len(predicted)*BATCH_SIZE
            tot0+=(0== tgt.squeeze()).sum().item()
            tot1+=(1== tgt.squeeze()).sum().item()
            tot2+=(2== tgt.squeeze()).sum().item()
            correct0+=((0== predicted) &(0==tgt.squeeze())).sum().item()
            correct1+=((1== predicted)&(1==tgt.squeeze())).sum().item()
            correct2+=((2== predicted)&(2==tgt.squeeze())).sum().item()
            
            conf00+=((0== predicted) &(0==tgt.squeeze())).sum().item()
            conf01+=((0== predicted)&(1==tgt.squeeze())).sum().item()
            conf02+=((0== predicted)&(2==tgt.squeeze())).sum().item()
            conf10+=((1== predicted) &(0==tgt.squeeze())).sum().item()
            conf11+=((1== predicted)&(1==tgt.squeeze())).sum().item()
            conf12+=((1== predicted)&(2==tgt.squeeze())).sum().item()
            conf20+=((2== predicted) &(0==tgt.squeeze())).sum().item()
            conf21+=((2== predicted)&(1==tgt.squeeze())).sum().item()
            conf22+=((2== predicted)&(2==tgt.squeeze())).sum().item()

            
            i+=targets.size()[0]
    etime=time.time()
    print("Time elapsed",etime-stime)
#     print(total,tot0,tot1,tot2)
#     print(correct)
    tp0=conf00
    fp0=conf01+conf02
    fn0=conf10+conf20
    if(tp0+fp0==0):
        prec0=0
    else:
        prec0=tp0/(tp0+fp0)
    if(tp0+fn0==0):
        reca0=0
    else:
        reca0=tp0/(tp0+fn0)
    
    tp1=conf11
    fp1=conf10+conf12
    fn1=conf01+conf21
    
    if(tp1+fp1==0):
        prec1=0
    else:
        prec1=tp1/(tp1+fp1)
    if(tp1+fn1==0):
        reca1=0
    else:
        reca1=tp1/(tp1+fn1)
    
    tp2=conf22
    fp2=conf20+conf21
    fn2=conf02+conf12
    
    if(tp2+fp2==0):
        prec2=0
    else:
        prec2=tp2/(tp2+fp2)
    if(tp2+fn2==0):
        reca2=0
    else:
        reca2=tp2/(tp2+fn2)
    
    prec=(prec0+prec1+prec2)/3
    reca=(reca0+reca1+reca2)/3
    if(prec+reca==0):
        f1sc=0
    else:
        f1sc=2*(prec*reca)/(prec+reca)
    print(total)
    print(correct)
    print("Acc:",correct/total)            
    print("Prec",prec)
    print("Recall",reca)
    print("F1",f1sc)
    return losses / Xtest_data.size(0),correct/total,prec,reca,f1sc,[conf00,conf01,conf02,conf10,conf11,conf12,conf20,conf21,conf22]

from torch.utils.data import DataLoader

def train_epoch(model, optimizer):
    model.train()
    losses = 0

    for batch, i in enumerate(range(0, Xtrain_data.size(0) - 1, BATCH_SIZE*bptt)):
        data, org_targets = get_batch(Xtrain_data, i,BATCH_SIZE)
        if (data.isnan().any() or data.isinf().any()):
#            print(data)
            continue
#         _,targets = get_batch(Ytrain_data,i)
        targets,_ = get_batch(Ytrain_data,i,BATCH_SIZE)
#         src = src.to(DEVICE)
#         tgt = tgt.to(DEVICE)
        targets=torch.unsqueeze(targets,1)
        src=batchify(data,BATCH_SIZE)
        tgt=batchify(targets,BATCH_SIZE)
        src=src[:-1]
        tgt=tgt[1:]
        logits = model(src)

        optimizer.zero_grad()

        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()
        
    return losses / Xtrain_data.size(0)


def evaluate(model):
    model.eval()
    losses = 0
    total=0
    correct=0
    correct1=0
    tot1=0
    correct2=0
    tot2=0
    correct0=0
    tot0=0
    with torch.no_grad():
        for i in range(0, Xtest_data.size(0) - 1, bptt):

            data, org_targets = get_batch(Xtest_data, i,BATCH_SIZE)
            if (data.isnan().any() or data.isinf().any()):
#            print(data)
                continue
            targets,_ = get_batch(Ytest_data,i,BATCH_SIZE)
            targets=torch.unsqueeze(targets,1)
            src=batchify(data,BATCH_SIZE)
            tgt=batchify(targets,BATCH_SIZE)
#             print(src.size())
#             print(tgt.size())
            logits = model(src)
#             print(logits)
#             print(logits.size())

            loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt.reshape(-1))
            if(loss.isnan()):
                print(src,tgt_input)
                break
#            print(loss)
            losses += loss.item()
#             output_flat = logits.view(-1)
            _,predicted=torch.max(logits,-1)
#             print(predicted)
#             print(logits.size())
#             print(tgt_out.size())
            
            correct += (tgt.squeeze() == predicted).sum().item()
            total+=len(predicted)*BATCH_SIZE
            tot0+=(0== tgt.squeeze()).sum().item()
            tot1+=(1== tgt.squeeze()).sum().item()
            tot2+=(2== tgt.squeeze()).sum().item()
            correct0+=((0== predicted) &(0==tgt.squeeze())).sum().item()
            correct1+=((1== predicted)&(1==tgt.squeeze())).sum().item()
            correct2+=((2== predicted)&(2==tgt.squeeze())).sum().item()
            
    print(total)
    print(correct)
    print("Acc:",correct/total*100)            
    print("Acc0:",correct0/tot0*100,correct0,tot0)     
    print("Acc1:",correct1/tot1*100,correct1,tot1)     
    print("Acc2:",correct2/tot2*100,correct2,tot2)     
    return losses / Xtest_data.size(0)

In [14]:
evaluate(lstm)

Time elapsed 0.03901362419128418
7296
667
Acc: 0.09141995614035088
Prec 0.030473318713450295
Recall 0.3333333333333333
F1 0.05584160073674077


(0.0008486893085333017,
 0.09141995614035088,
 0.030473318713450295,
 0.3333333333333333,
 0.05584160073674077,
 [0, 0, 0, 0, 0, 0, 1373, 5256, 667])

In [15]:
from timeit import default_timer as timer
NUM_EPOCHS = 1000
best_val_loss=100000000
for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss,_ = train_epoch(lstm, optimizer)
    end_time = timer()
    val_loss,acc,prec,reca,f1sc,confusion = evaluate(lstm)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_confusion=confusion
        best_acc=acc
        best_prec=prec
        best_reca=reca
        best_f1sc=f1sc
        best_model = lstm



8512
5184
Acc: 0.6090225563909775
Prec 0.2562345014738221
Recall 0.35199573507275467
F1 0.29657667862545356
Time elapsed 0.05794119834899902
7296
5256
Acc: 0.7203947368421053
Prec 0.24013157894736845
Recall 0.3333333333333333
F1 0.27915869980879543
Epoch: 1, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.224s
8512
5979
Acc: 0.7024201127819549
Prec 0.23414003759398497
Recall 0.3333333333333333
F1 0.2750672831412601
Time elapsed 0.05960440635681152
7296
5256
Acc: 0.7203947368421053
Prec 0.24013157894736845
Recall 0.3333333333333333
F1 0.27915869980879543
Epoch: 2, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.194s
8512
5979
Acc: 0.7024201127819549
Prec 0.23414003759398497
Recall 0.3333333333333333
F1 0.2750672831412601
Time elapsed 0.044858455657958984
7296
5256
Acc: 0.7203947368421053
Prec 0.24013157894736845
Recall 0.3333333333333333
F1 0.27915869980879543
Epoch: 3, Train loss: 0.000, Val loss: 0.001, Epoch time = 0.192s
8512
5979
Acc: 0.7024201127819549
Prec 0.234140037593984

KeyboardInterrupt: 

In [ ]:


from datetime import datetime
now = datetime.now()
now.strftime("%m/%d/%Y, %H:%M:%S")

date_time = now.strftime("%m_%d_%Y")
PATH='best_model_seq_attLSTM_'+date_time+str(MBR_NO)+'_'+str(BRN_NO)

In [ ]:
torch.save(best_model.state_dict(), PATH)
file_name='result_attLSTM_norm'+date_time+'.txt'
text_to_append=PATH+'\t'+"Acc:"+str(best_acc)+'\t'+"prec:"+str(best_prec)+'\t'+"recall:"+str(best_reca)+'\t'+"f1sc:"+str(best_f1sc)
print(text_to_append)
with open(file_name, "a+") as file_object:
    # Move read cursor to the start of file.
    file_object.seek(0)
    # If file is not empty then append '\n'
    data = file_object.read(100)
    if len(data) > 0:
        file_object.write("\n")
    # Append text at the end of file
    file_object.write(text_to_append)